In [1]:
import cobra
import itertools
import pandas as pd
from openpyxl import Workbook

In [ ]:
# Set the output txt file format
def output_txt(need_fluxes,outfile):
    with open(outfile,'w') as outf:
        for need_id in need_fluxes.index: # need_fluxes.index has the IDs of reactions with fluxes
            rea = model.reactions.get_by_id(need_id) # get the reaction object by ID
            need_fluxes[need_id] 
            fl = str(round(need_fluxes[need_id],5)) # round(x, n) returns float x rounded to n decimal places
            outline_list = [need_id,fl,rea.reaction,str(rea.lower_bound),str(rea.upper_bound),str(rea.objective_coefficient)]
            outf.write("\t".join(outline_list)+'\n')

In [ ]:
# Set the output Excel file format
def output_excel(need_fluxes, excel_file):
    data = []
    for need_id in need_fluxes.index:
        rea = model.reactions.get_by_id(need_id)
        fl = round(need_fluxes[need_id], 5)
        # extract ec_number and kegg_id from annotation
        ec_number = rea.annotation.get('ec_number', '')  
        kegg_id = rea.annotation.get('kegg_id', '')  
        data.append([need_id, fl, rea.reaction, rea.lower_bound, rea.upper_bound, rea.objective_coefficient, ec_number, kegg_id])
    
    df = pd.DataFrame(data, columns=['Reaction ID', 'Flux', 'Reaction Equation', 'Lower Bound', 'Upper Bound', 'Objective Coefficient', 'EC Number', 'KEGG id'])
    
    df.to_excel(excel_file, index=False)

# Calculation of carbon assimilation pathways involved in a single carbon fixation reaction

In [ ]:
inputdir = ".model/" 
outputdir = ".result/output(ACCOA)1/"   
model = cobra.io.read_sbml_model(inputdir + "Metacyc6578.xml") # model
# steps = 20    # limit number of steps
Cfixationslen = 49    # number of carbon fixation reactions to combine
a=range(0,Cfixationslen)
reactions_data = [
    ('R_EX_56', 'ATP_c + WATER_c --> ADP_c + Pi_c', (-1000, 1000), 0),
    ('R_EX_57', 'ATP_c + WATER_c --> AMP_c + PPI_c', (-1000, 1000), 0),
    ('R_EX_89', 'NADPH_c --> NADP_c + 2.0 PROTON_c', (-1000, 1000), 0),
    ('R_EX_90', 'NADH_c --> NAD_c + 2.0 PROTON_c', (-1000, 1000), 0),
    ('R_EX_901', 'NADH_c + Acceptor_c + PROTON_c --> NAD_c + Donor__45__H2_c', (-1000, 1000), 0),
    ('R_EX_902', 'NADH_c + Ox__45__Thioredoxin_c --> NAD_c + Red__45__Thioredoxin_c', (-1000, 1000), 0),
    ('R_EX_903', 'NADH_c + Oxidized__45__Amicyanins_c --> NAD_c + Reduced__45__Amicyanins_c', (-1000, 1000), 0),
    ('R_EX_904', 'NADH_c + ETF__45__Oxidized_c --> NAD_c + ETF__45__Reduced_c', (-1000, 1000), 0),
    ('R_EX_905', 'NADH_c + FAD_c + 2.0 PROTON_c --> NAD_c + FADH2_c', (-1000, 1000), 0),
    ('objACCOA_c', 'ACETYL__45__COA_c --> CO__45__A_c', (5, 5), 1)
    # ('objGLYOX_c', 'GLYOX_c --> ', (5, 5), 1)
    # ('objOXALATE_c', 'OXALATE_c --> ', (5, 5), 1)
    # ('objPYR_c', 'PYRUVATE_c --> ', (3.333333, 3.333333), 1)
    # ('objG3P_c', 'G3P_c --> ', (3.333333, 3.333333), 1)
]

for reaction_name, reaction_equation, bounds, object in reactions_data:
    reaction = cobra.Reaction(reaction_name)
    model.add_reaction(reaction)
    reaction.build_reaction_from_string(reaction_equation)
    reaction.lower_bound, reaction.upper_bound = bounds
    reaction.objective_coefficient = object

model.objective = 'objACCOA_c'
model_obj = 'objACCOA_c'

# combinations: select 1 reaction from the Cfixationslen reactions
closeidno1 = itertools.combinations(range(1, Cfixationslen+1), 1) 

wb = Workbook()
ws = wb.active

# write header row
ws.append(["Combination", "Target", "Reaction Number", "ATP Number", "NADPH Number", "NADH Number"])

for i in closeidno1:
    # enable each of the first 49 reactions one by one
    model.reactions[i[0]-1].bounds = (0, 10) 
    try:
        pfba_solution = cobra.flux_analysis.pfba(model)
        need_fluxes =pfba_solution.fluxes[abs(pfba_solution.fluxes) > 1e-10]

        # filtered_fluxes = {reaction_id: flux for reaction_id, flux in need_fluxes.items()
        #                    if not (reaction_id.startswith('obj') or reaction_id.startswith('R_EX_') or reaction_id.startswith('RXN0-5224'))}
        # print(len(filtered_fluxes))

        # filter out the following reactions during the counting of reaction steps
        filtered_fluxes = {reaction_id: flux for reaction_id, flux in need_fluxes.items()
                        if not (reaction_id.startswith('obj') or 
                                reaction_id.startswith('R_EX_') or 
                                reaction_id.startswith('RXN0-5224') or
                                reaction_id.startswith('RXN-12957') or
                                reaction_id.startswith('GLUTARYL-COA-DEHYDROG-RXN') or
                                reaction_id.startswith('PROPANOATECOA-LIGASE-RXN') or
                                reaction_id.startswith('ACETATE--COA-LIGASE-ADP-FORMING-RXN') or
                                reaction_id.startswith('RXN-16774') or
                                reaction_id.startswith('RXN-16767') or
                                reaction_id.startswith('RXN-14204') or
                                reaction_id.startswith('GLUCONATE-2-DEHYDROGENASE-RXN') or
                                reaction_id.startswith('HYDROG-RXN') or
                                reaction_id.startswith('RXN-12450') or
                                reaction_id.startswith('COENZYME-F420-HYDROGENASE-RXN'))}

        # check if the flux of the carbon fixation reaction is non-zero
        reaction1_flux = pfba_solution.fluxes.get(model.reactions[i[0] - 1].id, 0)
        print(reaction1_flux)

        if abs(reaction1_flux) < 1e-10:
            print(f"Reaction {i} has zero flux, skipping output")
        elif len(need_fluxes) == 0:
        # elif len(need_fluxes) == 0 or len(filtered_fluxes) > steps:
            print(f"Reaction {i} combination yielded no flux results, skipping output")
        else:

            outfile = outputdir + str(i) + '.txt'
            output_txt(need_fluxes, outfile)

            excel_file = outputdir + str(i) + '.xlsx'
            output_excel(need_fluxes, excel_file)

            # check if required fluxes exist
            r_ex_56_flux = need_fluxes.get('R_EX_56', 0)
            r_ex_57_flux = need_fluxes.get('R_EX_57', 0)
            r_ex_89_flux = need_fluxes.get('R_EX_89', 0)
            r_ex_90_flux = need_fluxes.get('R_EX_90', 0)
            
            # calculate combined ATP flux
            combined_flux = (r_ex_56_flux) * (-0.1) + (r_ex_57_flux) * (-0.2)

            # write to summary file with three decimal places
            ws.append([str(i), model.reactions.get_by_id(model_obj).reaction, len(filtered_fluxes), f"{combined_flux:.3f}", f"{(r_ex_89_flux) * (-0.1):.3f}", f"{(r_ex_90_flux) * (-0.1):.3f}"])
            print(f"Successfully processed reaction {i}")

    except Exception as e:
        print(f"Error processing reaction {i}: {e}")
    
    model.reactions[i[0]-1].bounds = (0.0, 0.0)

wb.save(outputdir + "summary.xlsx")

# Calculation of carbon assimilation pathways involved in two carbon fixation reactions

In [ ]:
inputdir = ".model/" 
outputdir = ".result/output(ACCOA)2/"   
model = cobra.io.read_sbml_model(inputdir + "Metacyc6578.xml") # model
# steps = 20    # limit number of steps
Cfixationslen = 49    # number of carbon fixation reactions to combine
a=range(0,Cfixationslen)
reactions_data = [
    ('R_EX_56', 'ATP_c + WATER_c --> ADP_c + Pi_c', (-1000, 1000), 0),
    ('R_EX_57', 'ATP_c + WATER_c --> AMP_c + PPI_c', (-1000, 1000), 0),
    ('R_EX_89', 'NADPH_c --> NADP_c + 2.0 PROTON_c', (-1000, 1000), 0),
    ('R_EX_90', 'NADH_c --> NAD_c + 2.0 PROTON_c', (-1000, 1000), 0),
    ('R_EX_901', 'NADH_c + Acceptor_c + PROTON_c --> NAD_c + Donor__45__H2_c', (-1000, 1000), 0),
    ('R_EX_902', 'NADH_c + Ox__45__Thioredoxin_c --> NAD_c + Red__45__Thioredoxin_c', (-1000, 1000), 0),
    ('R_EX_903', 'NADH_c + Oxidized__45__Amicyanins_c --> NAD_c + Reduced__45__Amicyanins_c', (-1000, 1000), 0),
    ('R_EX_904', 'NADH_c + ETF__45__Oxidized_c --> NAD_c + ETF__45__Reduced_c', (-1000, 1000), 0),
    ('R_EX_905', 'NADH_c + FAD_c + 2.0 PROTON_c --> NAD_c + FADH2_c', (-1000, 1000), 0),
    ('objACCOA_c', 'ACETYL__45__COA_c --> CO__45__A_c', (5, 5), 1)
    # ('objGLYOX_c', 'GLYOX_c --> ', (5, 5), 1)
    # ('objOXALATE_c', 'OXALATE_c --> ', (5, 5), 1)
    # ('objPYR_c', 'PYRUVATE_c --> ', (3.333333, 3.333333), 1)
    # ('objG3P_c', 'G3P_c --> ', (3.333333, 3.333333), 1)
]

for reaction_name, reaction_equation, bounds, object in reactions_data:
    reaction = cobra.Reaction(reaction_name)
    model.add_reaction(reaction)
    reaction.build_reaction_from_string(reaction_equation)
    reaction.lower_bound, reaction.upper_bound = bounds
    reaction.objective_coefficient = object

model.objective = 'objACCOA_c'
model_obj = 'objACCOA_c'

# combinations: select 2 reactions from the Cfixationslen reactions
closeidno2 = itertools.combinations(range(1, Cfixationslen + 1), 2) 

wb = Workbook()
ws = wb.active

# write header row
ws.append(["Combination", "Target", "Reaction Number", "ATP Number", "NADPH Number", "NADH Number"])
for i in closeidno2:
    # enable each of the first 49 reactions two by two
    model.reactions[i[0] - 1].bounds = (0, 5)
    model.reactions[i[1] - 1].bounds = (0, 5)
    try:
        pfba_solution = cobra.flux_analysis.pfba(model)
        need_fluxes =pfba_solution.fluxes[abs(pfba_solution.fluxes) > 1e-10]

        # filtered_fluxes = {reaction_id: flux for reaction_id, flux in need_fluxes.items()
        #                    if not (reaction_id.startswith('obj') or reaction_id.startswith('R_EX_') or reaction_id.startswith('RXN0-5224'))}
        # print(len(filtered_fluxes))

        # filter out the following reactions during the counting of reaction steps
        filtered_fluxes = {reaction_id: flux for reaction_id, flux in need_fluxes.items()
                        if not (reaction_id.startswith('obj') or 
                                reaction_id.startswith('R_EX_') or 
                                reaction_id.startswith('RXN0-5224') or
                                reaction_id.startswith('RXN-12957') or
                                reaction_id.startswith('GLUTARYL-COA-DEHYDROG-RXN') or
                                reaction_id.startswith('PROPANOATECOA-LIGASE-RXN') or
                                reaction_id.startswith('ACETATE--COA-LIGASE-ADP-FORMING-RXN') or
                                reaction_id.startswith('RXN-16774') or
                                reaction_id.startswith('RXN-16767') or
                                reaction_id.startswith('RXN-14204') or
                                reaction_id.startswith('GLUCONATE-2-DEHYDROGENASE-RXN') or
                                reaction_id.startswith('HYDROG-RXN') or
                                reaction_id.startswith('RXN-12450') or
                                reaction_id.startswith('COENZYME-F420-HYDROGENASE-RXN'))}

        # check if the flux of the carbon fixation reaction is non-zero
        reaction1_flux = pfba_solution.fluxes.get(model.reactions[i[0] - 1].id, 0)
        print(reaction1_flux)

        if abs(reaction1_flux) < 1e-10:
            print(f"Reaction {i} has zero flux, skipping output")
        elif len(need_fluxes) == 0:
        # elif len(need_fluxes) == 0 or len(filtered_fluxes) > steps:
            print(f"Reaction {i} combination yielded no flux results, skipping output")
        else:

            outfile = outputdir + str(i) + '.txt'
            output_txt(need_fluxes, outfile)

            excel_file = outputdir + str(i) + '.xlsx'
            output_excel(need_fluxes, excel_file)

            # check if required fluxes exist
            r_ex_56_flux = need_fluxes.get('R_EX_56', 0)
            r_ex_57_flux = need_fluxes.get('R_EX_57', 0)
            r_ex_89_flux = need_fluxes.get('R_EX_89', 0)
            r_ex_90_flux = need_fluxes.get('R_EX_90', 0)
            
            # calculate combined ATP flux
            combined_flux = (r_ex_56_flux) * (-0.1) + (r_ex_57_flux) * (-0.2)

            # write to summary file with three decimal places
            ws.append([str(i), model.reactions.get_by_id(model_obj).reaction, len(filtered_fluxes), f"{combined_flux:.3f}", f"{(r_ex_89_flux) * (-0.1):.3f}", f"{(r_ex_90_flux) * (-0.1):.3f}"])
            print(f"Successfully processed reaction {i}")

    except Exception as e:
        print(f"Error processing reaction {i}: {e}")
    
    model.reactions[i[0] - 1].bounds = (0, 0)
    model.reactions[i[1] - 1].bounds = (0, 0)

wb.save(outputdir + "summary.xlsx")

# Calculation of carbon assimilation pathways involved in three carbon fixation reactions

In [ ]:
inputdir = ".model/" 
outputdir = ".result/output(PYR)3/"   
model = cobra.io.read_sbml_model(inputdir + "Metacyc6578.xml") # model
# steps = 20    # limit number of steps
Cfixationslen = 49    # number of carbon fixation reactions to combine
a=range(0,Cfixationslen)

reactions_data = [
    ('R_EX_56', 'ATP_c + WATER_c --> ADP_c + Pi_c', (-1000, 1000), 0),
    ('R_EX_57', 'ATP_c + WATER_c --> AMP_c + PPI_c', (-1000, 1000), 0),
    ('R_EX_89', 'NADPH_c --> NADP_c + 2.0 PROTON_c', (-1000, 1000), 0),
    ('R_EX_90', 'NADH_c --> NAD_c + 2.0 PROTON_c', (-1000, 1000), 0),
    ('R_EX_901', 'NADH_c + Acceptor_c + PROTON_c --> NAD_c + Donor__45__H2_c', (-1000, 1000), 0),
    ('R_EX_902', 'NADH_c + Ox__45__Thioredoxin_c --> NAD_c + Red__45__Thioredoxin_c', (-1000, 1000), 0),
    ('R_EX_903', 'NADH_c + Oxidized__45__Amicyanins_c --> NAD_c + Reduced__45__Amicyanins_c', (-1000, 1000), 0),
    ('R_EX_904', 'NADH_c + ETF__45__Oxidized_c --> NAD_c + ETF__45__Reduced_c', (-1000, 1000), 0),
    ('R_EX_905', 'NADH_c + FAD_c + 2.0 PROTON_c --> NAD_c + FADH2_c', (-1000, 1000), 0),
    ('objPYR_c', 'PYRUVATE_c --> ', (3.333333, 3.333333), 1)
    # ('objG3P_c', 'G3P_c --> ', (3.33333, 3.333333), 1)
]

for reaction_name, reaction_equation, bounds, object in reactions_data:
    reaction = cobra.Reaction(reaction_name)
    model.add_reaction(reaction)
    reaction.build_reaction_from_string(reaction_equation)
    reaction.lower_bound, reaction.upper_bound = bounds
    reaction.objective_coefficient = object

model.objective = 'objPYR_c'
model_obj = 'objPYR_c'

# combinations: select 3 reactions from the Cfixationslen reactions
closeidno3 = itertools.combinations(range(1, Cfixationslen + 1), 3)

wb = Workbook()
ws = wb.active

# write header row
ws.append(["Combination", "Target", "Reaction Number", "ATP Number", "NADPH Number", "NADH Number"])
for i in closeidno3:
    # enable each of the first 49 reactions three by three
    model.reactions[i[0] - 1].bounds = (0, 3.333333)
    model.reactions[i[1] - 1].bounds = (0, 3.333333)
    model.reactions[i[2] - 1].bounds = (0, 3.333333)
    try:
        pfba_solution = cobra.flux_analysis.pfba(model)
        need_fluxes =pfba_solution.fluxes[abs(pfba_solution.fluxes) > 1e-10]

        # filtered_fluxes = {reaction_id: flux for reaction_id, flux in need_fluxes.items()
        #                    if not (reaction_id.startswith('obj') or reaction_id.startswith('R_EX_') or reaction_id.startswith('RXN0-5224'))}
        # print(len(filtered_fluxes))

        # filter out the following reactions during the counting of reaction steps
        filtered_fluxes = {reaction_id: flux for reaction_id, flux in need_fluxes.items()
                        if not (reaction_id.startswith('obj') or 
                                reaction_id.startswith('R_EX_') or 
                                reaction_id.startswith('RXN0-5224') or
                                reaction_id.startswith('RXN-12957') or
                                reaction_id.startswith('GLUTARYL-COA-DEHYDROG-RXN') or
                                reaction_id.startswith('PROPANOATECOA-LIGASE-RXN') or
                                reaction_id.startswith('ACETATE--COA-LIGASE-ADP-FORMING-RXN') or
                                reaction_id.startswith('RXN-16774') or
                                reaction_id.startswith('RXN-16767') or
                                reaction_id.startswith('RXN-14204') or
                                reaction_id.startswith('GLUCONATE-2-DEHYDROGENASE-RXN') or
                                reaction_id.startswith('HYDROG-RXN') or
                                reaction_id.startswith('RXN-12450') or
                                reaction_id.startswith('COENZYME-F420-HYDROGENASE-RXN'))}

        # check if the flux of the carbon fixation reaction is non-zero
        reaction1_flux = pfba_solution.fluxes.get(model.reactions[i[0] - 1].id, 0)
        print(reaction1_flux)

        if abs(reaction1_flux) < 1e-10:
            print(f"Reaction {i} has zero flux, skipping output")
        elif len(need_fluxes) == 0:
        # elif len(need_fluxes) == 0 or len(filtered_fluxes) > steps:
            print(f"Reaction {i} combination yielded no flux results, skipping output")
        else:

            outfile = outputdir + str(i) + '.txt'
            output_txt(need_fluxes, outfile)

            excel_file = outputdir + str(i) + '.xlsx'
            output_excel(need_fluxes, excel_file)

            # check if required fluxes exist
            r_ex_56_flux = need_fluxes.get('R_EX_56', 0)
            r_ex_57_flux = need_fluxes.get('R_EX_57', 0)
            r_ex_89_flux = need_fluxes.get('R_EX_89', 0)
            r_ex_90_flux = need_fluxes.get('R_EX_90', 0)
            
            # calculate combined ATP flux
            combined_flux = (r_ex_56_flux) * (-0.1) + (r_ex_57_flux) * (-0.2)

            # write to summary file with three decimal places
            ws.append([str(i), model.reactions.get_by_id(model_obj).reaction, len(filtered_fluxes), f"{combined_flux:.3f}", f"{(r_ex_89_flux) * (-0.1):.3f}", f"{(r_ex_90_flux) * (-0.1):.3f}"])
            print(f"Successfully processed reaction {i}")

    except Exception as e:
        print(f"Error processing reaction {i}: {e}")
    
    model.reactions[i[0] - 1].bounds = (0, 0)
    model.reactions[i[1] - 1].bounds = (0, 0)
    model.reactions[i[2] - 1].bounds = (0, 0)

wb.save(outputdir + "summary.xlsx")